# Self Organising Map Challenge

## The Kohonen Network

The Kohonen Self Organising Map (SOM) provides a data visualization technique which helps to understand high dimensional data by reducing the dimensions of data to a map. SOM also represents clustering concept by grouping similar data together.

Unlike other learning technique in neural networks, training a SOM requires no target vector. A SOM learns to classify the training data without any external supervision.

![Network](kohonen.png)

### Structure
A network has a width and a height that descibes the grid of nodes.  For example, the grid may be 4x4, and so there would be 16 nodes.

Each node has a weight for each value in the input vector.  A weight is simply a float value that the node multiplies the input value by to determine how influential it is (see below)

Each node has a set of weights that match the size of the input vector.  For example, if the input vector has 10 elements, each node would have 10 weights.

### Training 
To train the network

1. Each node's weights are initialized.
2. We enumerate through the training data for some number of iterations (repeating if necessary).  The current value we are training against will be referred to as the `current input vector`
3. Every node is examined to calculate which one's weights are most like the input vector. The winning node is commonly known as the Best Matching Unit (BMU).
4. The radius of the neighbourhood of the BMU is now calculated. This is a value that starts large, typically set to the 'radius' of the lattice,  but diminishes each time-step. Any nodes found within this radius are deemed to be inside the BMU's neighbourhood.
5. Each neighbouring node's (the nodes found in step 4) weights are adjusted to make them more like the input vector. The closer a node is to the BMU, the more its weights get altered.
6. Go to step 2 until we've completed N iterations.
    

### Calculating the Best Matching Unit (BMU)

To determine the best matching unit, one method is to iterate through all the nodes and calculate the Euclidean distance between each node's weight vector and the current input vector. The node with a weight vector closest to the input vector is tagged as the BMU.

The Euclidean distance $\mathsf{distance}_{i}$ (from the input vector $V$ to the $i$th node's weights $W_i$)is given as (using Pythagoras):

$$ \mathsf{distance}_{i}=\sqrt{\sum_{k=0}^{k=n}(V_k - W_{i_k})^2}$$

where V is the current input vector and $W_i$ is the node's weight vector.  $n$ is the size of the input & weight vector.

*Note*: $V$ and $W$ are vectors.  $V$ is the input vector, and $W_i$ is the weight vector of the $i$th node.  $V_k$ and $W_{i_k}$ represent the $k$'th value within those vectors.  

The BMU is the node with the minimal distance for the current input vector

### Calculating the Neighbourhood Radius

The next step is to calculate which of the other nodes are within the BMU's neighbourhood. All these nodes will have their weight vectors altered.


First we calculate what the radius of the neighbourhood should be and then use Pythagoras to determine if each node is within the radial distance or not.

A unique feature of the Kohonen learning algorithm is that the area of the neighbourhood shrinks over time. To do this we use the exponential decay function:

Given a desired number of training iterations $n$:
$$n_{\mathsf{max iterations}} = 100$$

Calculate the radius $\sigma_t$ at iteration number $t$:

$$\sigma_t = \sigma_0 \exp\left(- \frac{t}{\lambda} \right) \qquad t = 1,2,3,4... $$

Where $\sigma_0$ denotes the neighbourhood radius at iteration $t=0$, $t$ is the current iteration. We define $\sigma_0$ (the initial radius) and $\lambda$ (the time constant) as below:

$$\sigma_0 = \frac{\max(width,height)}{2} \qquad \lambda = \frac{n_{\mathsf{max iterations}}}{\log(\sigma_0)} $$

Where $width$ & $height$ are the width and height of the grid.

### Calculating the Learning Rate

We define the initial leanring rate $\alpha_0$ at iteration $t = 0$ as:
$$\alpha_0 = 0.1$$

So, we can calculate the learning rate at a given iteration t as:

$$\alpha_t = \alpha_0 \exp \left(- \frac{t}{\lambda} \right) $$

where $t$ is the iteration number, $\lambda$ is the time constant (calculated above)
        
### Calculating the Influence

As well as the learning rate, we need to calculate the influence $\theta_t$ of the learning/training at a given iteration $t$.  

So for each node, we need to caclulate the euclidean distance $d_i$ from the BMU to that node.  Similar to when we calculate the distance to find the BMU, we use Pythagoras.  The current ($i$th) node's x position is given by $x(W_i)$, and the BMU's x position is, likewise, given by $x(Z)$.  Similarly, $y()$ returns the y position of a node.

$$ d_{i}=\sqrt{(x(W_i) - x(Z))^2 + (y(W_i) - y(Z))^2} $$

Then, the influence decays over time according to:

$$\theta_t = \exp \left( - \frac{d_{i}^2}{2\sigma_t^2} \right) $$

Where $\sigma_t$ is the neighbourhood radius at iteration $t$ as calculated above. 

Note: You will need to come up with an approach to x() and y().


### Updating the Weights

To update the weights of a given node, we use:

$$W_{i_{t+1}} = W_{i_t} + \alpha_t \theta_t (V_t - W_{i_t})$$
        
So $W_{i_{t+1}}$ is the new value of the weight for the $i$th node, $V_t$ is the current value of the training data, $W_{i_t}$ is the current weight and $\alpha_t$ and $\theta_t$ are the learning rate and influence calculated above.

*Note*: the $W$ and $V$ are vectors 

## Challenge

Sam has written an implementation of a Self Organising Map. Consider the following criteria when assessing Sam's code:

- Could the code be made more efficient? A literal interpretation of the instructions above is not necessary.
- Is the code best structured for later use by other developers and in anticipation of productionisation?
- How would you approach productionising this application?
- Anything else you think is relevant.

### Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import time

### Test Data

In [ ]:
input_data = np.random.random((10,3))

### Original

In [ ]:
# kohonen.py

def train(input_data, n_max_iterations, width, height):
    σ0 = max(width, height) / 2
    α0 = 0.1
    weights = np.random.random((width, height, 3))
    λ = n_max_iterations / np.log(σ0)
    for t in range(n_max_iterations):
        if t % 10 == 0:
            print(f"Iteration: {t}")
        σt = σ0 * np.exp(-t/λ)
        αt = α0 * np.exp(-t/λ)
        for vt in input_data:
            bmu = np.argmin(np.sum((weights - vt) ** 2, axis=2))
            bmu_x, bmu_y = np.unravel_index(bmu, (width, height))
            for x in range(width):
                for y in range(height):
                    di = np.sqrt(((x - bmu_x) ** 2) + ((y - bmu_y) ** 2))
                    θt = np.exp(-(di ** 2) / (2*(σt ** 2)))
                    weights[x, y] += αt * θt * (vt - weights[x, y])
    return weights


### Updated

In [ ]:
# kohonen_update.py

class KohonenTrainer:

    def __init__(self, n_max_iterations, width, height):

        self.n_max_iterations = n_max_iterations
        self.width = width
        self.height = height

        self.radius_0 = max(self.width, self.height) / 2
        self.lr_0 = 0.1
        self.lambda_val = self.n_max_iterations / np.log(self.radius_0)

    def calculate_bmu(self, weights, row_vector):
        bmu = np.argmin(np.sum((weights - row_vector) ** 2, axis=2))
        return np.unravel_index(bmu, (self.width, self.height))
    
    def influence(self, dist, radius_t):
        return np.exp(-(dist ** 2) / (2*(radius_t ** 2)))

    def euclidean_distance(self, point1, point2):
        return np.sqrt(((point1[0] - point2[0]) ** 2) + ((point1[1] - point2[1]) ** 2))
    
    def neighbour_radius(self, t):
        return self.radius_0 * np.exp(-t/self.lambda_val)

    def learning_rate(self, t):
        return self.lr_0 * np.exp(-t/self.lambda_val)
    
    def train(self, input_data):
        weights = np.random.random((self.width, self.height, 3))
        
        for t in range(self.n_max_iterations):
            if t % 100 == 0:
                print(f"Iteration: {t}/{self.n_max_iterations}")
                
            lr_t = self.learning_rate(t)
            radius_t = self.neighbour_radius(t)
            
            for vt in input_data:    
                bmu_x, bmu_y = self.calculate_bmu(weights, vt)

                # Bounding box around the neighbourhood radius
                window_top_left = [max(bmu_x - int(radius_t), 0), max(bmu_y - int(radius_t), 0)]
                window_bottom_right = [min(bmu_x + int(radius_t), self.width), min(bmu_y + int(radius_t), self.height)]
                
                for x in range(window_top_left[0], window_bottom_right[0]):
                    for y in range(window_top_left[1], window_bottom_right[1]):
                        
                        distance = self.euclidean_distance([x,y], [bmu_x, bmu_y])
                        if distance <= radius_t:
                            influence_t = self.influence(distance, radius_t)
                            weights[x, y] += lr_t * influence_t * (vt - weights[x, y])
        return weights


In [ ]:
start = time.time()
image_data_OG_small = train(input_data, 100, 10, 10)
end = time.time()
time_OG_small = end - start

start = time.time()
image_data_OG_big = train(input_data, 1000, 100, 100)
end = time.time()
time_OG_big = end - start

start = time.time()
kohonen_trainer = KohonenTrainer(100, 10, 10)
image_data_small = kohonen_trainer.train(input_data)
end = time.time()
time_small = end - start

start = time.time()
kohonen_trainer = KohonenTrainer(1000, 100, 100)
image_data_big = kohonen_trainer.train(input_data)
end = time.time()
time_big = end - start

### OG Output

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,11))

axes[0,0].imshow(image_data_OG_small)
axes[0,0].set_title("Original Program: 100 Iterations, 10x10")
axes[0,0].set_xlabel(f"Time: {round(time_OG_small, 1)} Seconds")

axes[0,1].imshow(image_data_OG_big)
axes[0,1].set_title("Original Program: 1000 Iterations, 100x100")
axes[0,1].set_xlabel(f"Time: {round(time_OG_big, 1)} Seconds")

axes[1,0].imshow(image_data_small)
axes[1,0].set_title("Updated Program: 100 Iterations, 10x10")
axes[1,0].set_xlabel(f"Time: {round(time_small, 2)} Seconds")

axes[1,1].imshow(image_data_big)
axes[1,1].set_title("Updated Program: 1000 Iterations, 100x100")
axes[1,1].set_xlabel(f"Time: {round(time_big, 2)} Seconds")

plt.show()

### 100 Rows Just For Fun!

In [ ]:
input_data = np.random.random((100,3))

start = time.time()
kohonen_trainer = KohonenTrainer(100, 10, 10)
image_data_100 = kohonen_trainer.train(input_data)
end = time.time()
time_100 = end - start

start = time.time()
kohonen_trainer = KohonenTrainer(1000, 100, 100)
image_data_1000 = kohonen_trainer.train(input_data)
end = time.time()
time_1000 = end - start

fig, axes = plt.subplots(1,2, figsize=(10,5))

axes[0].imshow(image_data_100)
axes[0].set_title("Updated Program: 100 Iterations, 10x10")
axes[0].set_xlabel(f"Time: {round(time_100, 2)} Seconds")

axes[1].imshow(image_data_1000)
axes[1].set_title("Updated Program: 1000 Iterations, 100x100")
axes[1].set_xlabel(f"Time: {round(time_1000, 2)} Seconds")

plt.show()